In [42]:
import pandas as pd
import requests
from datetime import datetime


pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 2)
GET_SKU_BY_EAN_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pvt/sku/stockkeepingunitbyean/"
)
GET_FIELD_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pub/specification/fieldGet/"
)
GET_FIELD_VALUES_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pub/specification/fieldvalue/"
)
GET_CATEGORY_FIELD_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pub/specification/field/listByCategoryId/"
)
DELETE_PRODUCT_SPECIFICATIONS_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog/pvt/product/{product_id}/specification"
)
API_APP_KEY_NAME = ""
API_APP_KEY_VALUE = ""
API_APP_TOKEN_NAME = ""
API_APP_TOKEN_VALUE = ""

In [43]:
"""
Leer excel products-master y convertirlo a csv
"""
products_master_excel_df = pd.read_excel("source-files/products-master-for-accesories-buy-together-suggested.xlsx", sheet_name=1)
products_master_excel_df.to_csv("source-files/products-master-for-accesories-buy-together-suggested.csv", encoding="utf-8")
products_master_csv_df = pd.read_csv("source-files/products-master-for-accesories-buy-together-suggested.csv", encoding="utf-8")

In [44]:
log_clean_excel_df = pd.read_excel("source-files/products-master-for-accesories-buy-together-suggested-log-clean.xls")
log_clean_excel_df.to_csv("source-files/products-master-for-accesories-buy-together-suggested-log-clean.csv", encoding="utf-8")
log_csv_df = pd.read_csv("source-files/products-master-for-accesories-buy-together-suggested-log-clean.csv", encoding="utf-8")

In [45]:
api_request_headers = {
    "accept": "application/json",
    "content-type": "application/json",
    API_APP_KEY_NAME: API_APP_KEY_VALUE,
    API_APP_TOKEN_NAME: API_APP_TOKEN_VALUE,
}
for index, row in products_master_csv_df[1:].iterrows():  
    print("ean", row['EAN'])
        
    log_row = {
        "product_id": None,
        "ean": row['EAN'],
        "recommended_ean": None,
        "recommended_product_id": None,
        "missing_recommended_ean": None,
        "buy_together_ean": None,
        "buy_together_product_id": None,
        "missing_buy_together_ean": None,
    }
    if isinstance(row["recommended"], str):

        recommended_ean = [ean.strip() for ean in row["recommended"].replace("-", ",").split(",")]
        log_row["recommended_ean"] = ", ".join([str(ean) for ean in recommended_ean])
        print("recommended", log_row["recommended_ean"])
        
        recommended_product_id = []
        missing_recommended_ean = []
        for ean in [r_ean for r_ean in recommended_ean if r_ean]:
            sku = requests.request(
                "GET",
                f"{GET_SKU_BY_EAN_ENDPOINT}{ean}",
                headers=api_request_headers
            ).json()
            if isinstance(sku, str) and sku == "SKU not found.":
                missing_recommended_ean.append(ean)
                continue
            recommended_product_id.append(sku["ProductId"])
        log_row["recommended_product_id"] = ", ".join([str(ean) for ean in recommended_product_id])
        log_row["missing_recommended_ean"] = ", ".join([str(ean) for ean in missing_recommended_ean])
        products_master_csv_df.loc[products_master_csv_df["EAN"] == row["EAN"], "recommended"] = ", ".join([str(ean) for ean in recommended_product_id])

    if isinstance(row["buy_together"], str):

        buy_together_ean = [ean.strip() for ean in row["buy_together"].replace("-", ",").split(",")]
        log_row["buy_together_ean"] = ", ".join([str(ean) for ean in buy_together_ean])
        print("buy_together", log_row["buy_together_ean"])

        buy_together_product_id = []
        missing_buy_together_ean = []
        for ean in [bt_ean for bt_ean in buy_together_ean if bt_ean]:
            sku = requests.request(
                "GET",
                f"{GET_SKU_BY_EAN_ENDPOINT}{ean}",
                headers=api_request_headers
            ).json()
            if isinstance(sku, str) and sku == "SKU not found.":
                missing_buy_together_ean.append(ean)
                continue
            buy_together_product_id.append(sku["ProductId"])
        
        log_row["buy_together_product_id"] = ", ".join([str(ean) for ean in buy_together_product_id])
        log_row["missing_buy_together_ean"] = ", ".join([str(ean) for ean in missing_buy_together_ean])
        products_master_csv_df.loc[products_master_csv_df["EAN"] == row["EAN"], "buy_together"] = ", ".join([str(ean) for ean in buy_together_product_id])

    sku = requests.request(
        "GET",
        f"{GET_SKU_BY_EAN_ENDPOINT}{row['EAN']}",
        headers=api_request_headers
    ).json()
    if isinstance(sku, str) and sku == "SKU not found.":
        log_row["product_id"] = sku
    else:
        log_row["product_id"] = sku["ProductId"]

    log_csv_df = log_csv_df.append(log_row, ignore_index=True)  

ean 7707172626948
recommended 7707172626955, 7707172626962
buy_together 7707172629284, 7707061693433
ean 7707061694225
recommended 7707061694249, 7707061689450
buy_together 7707172629284, 7705191040158
ean 7707061694249
recommended 7707061694225, 7707061689450
buy_together 7707172629284, 7705191040158
ean 7705191002736
recommended 7707061694225, 7707061689450
buy_together 7707172629284, 7707061695178
ean 7707061694256
recommended 7707061694263, 7707061611147
buy_together 7707061693259, 7707061695178
ean 7707061694263
recommended 7707061694263, 7707061611147
buy_together 7707061693259, 7707061695178
ean 7707061694270
recommended 7707061694263, 7707061611147
buy_together 7707061693259, 7707061695178
ean 7707061694287
recommended 7707061694263, 7707061611147
buy_together 7707061693259, 7707061695178
ean 7707061694294
recommended 7707172627006, 7707061695017
buy_together 7707061660060, 7707061699732
ean 7707061694300
recommended 7707172627013, 7705191003528
buy_together 7707172625194, 7707

In [46]:
products_master_csv_df.loc[products_master_csv_df["EAN"] == "7707172625262", "buy_together"]
# products_master_csv_df.loc[products_master_csv_df["EAN"] == "7707172625262", "recommended"]

66    22
Name: buy_together, dtype: object

In [47]:
requests.request(
                "GET",
                f"{GET_SKU_BY_EAN_ENDPOINT}7707061699770",
                headers=api_request_headers
            ).json()

{&#39;Id&#39;: 64,
 &#39;ProductId&#39;: 64,
 &#39;NameComplete&#39;: &#39;Horno microondas&#39;,
 &#39;ProductName&#39;: &#39;Horno microondas&#39;,
 &#39;ProductDescription&#39;: &#39;Horno microondas con 16 métodos automáticos de cocción, cuenta con motor axial que reparte de manera uniforme el calor generado por las resistencias.&#39;,
 &#39;ProductRefId&#39;: &#39;1.8045.73&#39;,
 &#39;TaxCode&#39;: None,
 &#39;SkuName&#39;: &#39;Horno microondas&#39;,
 &#39;IsActive&#39;: True,
 &#39;IsTransported&#39;: True,
 &#39;IsInventoried&#39;: True,
 &#39;IsGiftCardRecharge&#39;: False,
 &#39;ImageUrl&#39;: &#39;https://challengerco.vteximg.com.br/arquivos/ids/155671-176-176/7707061699770.jpg?v=637369165147000000&#39;,
 &#39;DetailUrl&#39;: &#39;/hm8045aceroinox-7707061699770/p&#39;,
 &#39;CSCIdentification&#39;: None,
 &#39;BrandId&#39;: &#39;2000001&#39;,
 &#39;BrandName&#39;: &#39;Challenger&#39;,
 &#39;IsBrandActive&#39;: True,
 &#39;Dimension&#39;: {&#39;cubicweight&#39;: 43.55,
  &#

In [48]:
products_master_csv_df.to_excel("generated-files/products-master-for-accesories-buy-together-suggested.xls", index=False)

In [49]:
now = str(datetime.now()).replace(" ", "-").replace(":", "_").split(".")[0]
log_csv_df.to_excel("generated-files/log-" + now + ".xls", index=False)

In [50]:
datetime.now()

datetime.datetime(2020, 10, 6, 11, 43, 3, 723181)